In [53]:
import pandas as pd
import pickle

In [54]:
tags=pd.read_csv('Data/ml-small/tags.csv')
tags.drop(columns=["timestamp",],inplace=True)
ratings=pd.read_csv('Data/ml-small/ratings.csv')
ratings.drop(columns=["timestamp"],inplace=True)
genre=pd.read_csv('Data/ml-small/movies.csv')
plots=pd.read_csv('Data/ml-small/plots.csv')

In [55]:
tag_final=pd.DataFrame.from_dict(tags.groupby("movieId")['tag'].apply(set).apply(list))
tag_final.head()

,tag
movieId,
1,"[pixar, fun]"
2,"[fantasy, game, magic board game, Robin Williams]"
3,"[old, moldy]"
5,"[pregnancy, remake]"
7,[remake]


In [56]:
rating_final=pd.DataFrame.from_dict(ratings.groupby('movieId')['userId'].count())
rating_final.columns=['votes_counts']
rating_final['votes_avg']=pd.Series(ratings.groupby('movieId')['rating'].mean())
rating_final.head()

,votes_counts,votes_avg
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429


In [57]:
C=rating_final['votes_avg'].mean()
m=rating_final['votes_counts'].quantile(0.95)
def weighted_rating(x):
    v=x['votes_counts']
    R=x['votes_avg']
    return (v/(v+m)*R)+(m/(v+m))*C
rating_final['weighted_rating']=rating_final.apply(weighted_rating,axis=1)
rating_final.drop(columns=['votes_avg'],inplace=True)
rating_final.head()

,votes_counts,weighted_rating
movieId,,
1,215,3.802806
2,110,3.381115
3,52,3.260960
4,7,3.145094
5,49,3.164949


In [58]:
genre['release_year']=genre['title'].str.extract('(\d\d\d\d)', expand=True)
genre["title_movie"]= genre["title"].str.split("(", n = 1, expand = True)[0] 
genre['genres']=genre['genres'].apply(lambda x: x.split('|'))
genre.drop(columns=["title"],inplace=True)
genre.head()

,movieId,genres,release_year,title_movie
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,Toy Story
1,2,"[Adventure, Children, Fantasy]",1995,Jumanji
2,3,"[Comedy, Romance]",1995,Grumpier Old Men
3,4,"[Comedy, Drama, Romance]",1995,Waiting to Exhale
4,5,[Comedy],1995,Father of the Bride Part II


In [59]:
df_merge_col = pd.merge(tag_final, rating_final, on='movieId',how='outer')

In [60]:
df_merge_col.head()

,tag,votes_counts,weighted_rating
movieId,,,
1,"[pixar, fun]",215.0,3.802806
2,"[fantasy, game, magic board game, Robin Williams]",110.0,3.381115
3,"[old, moldy]",52.0,3.260960
5,"[pregnancy, remake]",49.0,3.164949
7,[remake],54.0,3.221139


In [61]:
merge_genre=pd.merge(df_merge_col, genre, on='movieId',how='outer')

In [62]:
merge_final=pd.merge(merge_genre, plots, on='movieId',how='outer')
merge_final.head()

,movieId,tag,votes_counts,weighted_rating,genres,release_year,title_movie,imdb,tmdb
0,1,"[pixar, fun]",215.0,3.802806,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,Toy Story,A cowboy doll is profoundly threatened and jea...,"Led by Woody, Andy's toys live happily in his ..."
1,2,"[fantasy, game, magic board game, Robin Williams]",110.0,3.381115,"[Adventure, Children, Fantasy]",1995,Jumanji,When two kids find and play a magical board ga...,When siblings Judy and Peter discover an encha...
2,3,"[old, moldy]",52.0,3.260960,"[Comedy, Romance]",1995,Grumpier Old Men,John and Max resolve to save their beloved bai...,A family wedding reignites the ancient feud be...
3,5,"[pregnancy, remake]",49.0,3.164949,[Comedy],1995,Father of the Bride Part II,George Banks must deal not only with the pregn...,Just when George Banks has recovered from his ...
4,7,[remake],54.0,3.221139,"[Comedy, Romance]",1995,Sabrina,An ugly duckling having undergone a remarkable...,An ugly duckling having undergone a remarkable...


In [63]:
print(tag_final.info(),'\n','-----'*10)
print(rating_final.info(),'\n','-----'*10)
print(genre.info(),'\n','-----'*10)
print(merge_genre.info(),'\n','-----'*10)
print(merge_final.info(),'\n','-----'*10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 1 to 193565
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tag     1572 non-null   object
dtypes: object(1)
memory usage: 24.6+ KB
None 
 --------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 1 to 193609
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   votes_counts     9724 non-null   int64  
 1   weighted_rating  9724 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 227.9 KB
None 
 --------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       9742 non-null   int64 
 1   genres        9742 non-null   object
 2   rele

In [64]:
merge_final.isna().sum()

movieId               0
tag                8170
votes_counts         18
weighted_rating      18
genres                0
release_year         12
title_movie           0
imdb                  0
tmdb                  1
dtype: int64

In [65]:
merge_final.to_pickle('merged_data.pickle')

In [66]:
merge_final.to_csv('Data/ml-small/data.csv',index=False)